<a href="https://colab.research.google.com/github/fellowship/deep-and-wide-bandit/blob/dev/EDA_LargeSet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#!pip install -U ipykernel
#!pip install modin[dask]

In [2]:
%load_ext autoreload
%autoreload 2
%env JOBLIB_TEMP_FOLDER=/tmp

env: JOBLIB_TEMP_FOLDER=/tmp


In [3]:
import sys

!{sys.executable} -m pip install -U pandas-profiling[notebook]
!jupyter nbextension enable --py widgetsnbextension

Requirement already up-to-date: pandas-profiling[notebook] in /usr/local/lib/python3.6/dist-packages (2.10.0)
Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


# Mount Drive

In [4]:
from google.colab import drive
drive.mount("/content/GDrive")

Mounted at /content/GDrive


In [5]:
import os
import pandas as pd
from pandas_profiling import ProfileReport
from zipfile import ZipFile

#Formatting .describe() calls s.t. floats are displayed in non-scientific notation
pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [6]:
os.chdir(os.getcwd())
zip_path_l = ["/content/GDrive/MyDrive/July_Dec_2019.zip",
               "/content/GDrive/MyDrive/Jan_June_2020.zip", 
               "/content/GDrive/MyDrive/Jul_Sep_2020.zip"]

folder_name_l = [os.path.split(zip_path)[-1][:-4] for zip_path in zip_path_l]

for (zip_path, folder_name) in zip(zip_path_l, folder_name_l):
  
  if not(os.path.exists(folder_name)):
    
    with ZipFile(zip_path, 'r') as zip_obj:

      zip_obj.extractall(folder_name)

In [7]:
file_name_l = ['/content/July_Dec_2019/sends_july_dec_2019_2000',
               '/content/Jan_June_2020/sends_jan_june_2020_2000',
               '/content/Jul_Sep_2020/sends_july_sept_2020_2000']

col_names = ['riid','send_dt','launch_id','sub_sourcedev','opened','unsub',
             'rev_3dv2','aq_dt','aq_mo','aq_dow','aq_period','sends_since_last_open',
             'wk1_opens','wk4_opens','mo3_opens','mo6_opens','yr1_opens',
             'wk1_clicks','wk4_clicks','mo3_clicks','mo6_clicks',
             'rev_dec','rev_raw','snds_dec','snds_opens_dec','wk1_vis',
             'wk4_vis','mo3_vis','mo6_vis','wk1_pv','wk4_pv','mo3_pv','mo6_pv',
             'wk1_secs','wk4_secs','mo3_secs','mo6_secs','prev_optouts',
             'last_order','last_click','last_cart','last_visit','last_ded_open',
             'sent_time','offer_signature_id','dynamic_content_signature_id',
             'message_size','open_cnt','first_open_time','last_open_time',
             'hrs_to_first_open','hrs_to_last_open','click_cnt','first_click_time',
             'last_click_time','offer_category','hrs_to_first_click','hrs_to_last_click',
             'opt_out_cnt','first_opt_out_time','last_opt_out_time','opt_out_source',
             'opt_out_reason','hrs_to_first_opt_out','hrs_to_last_opt_out','campaign_id',
             'subject','marketing_strategy','campaign_type']

#os.remove('/content/July_Dec_2019/sends_july_dec_2019_2000')
#df2_jan_jun_20 = pd.read_csv(file_name_l[1], sep='\t', header=None, names=col_names)
#os.remove('/content/Jan_June_2020/sends_jan_june_2020_2000')
#df3_jul_sep_20 = pd.read_csv(file_name_l[2], sep='\t', header=None, names=col_names)
#os.remove('/content/Jul_Sep_2020/sends_july_sept_2020_2000')

In [13]:
#Inspect the first 1e5 rows of one of the dataframes
df = pd.read_csv(file_name_l[0], sep="\t", header=None, names=col_names, nrows=1e5)
df.head(10)

,riid,send_dt,launch_id,sub_sourcedev,opened,unsub,rev_3dv2,aq_dt,aq_mo,aq_dow,aq_period,sends_since_last_open,wk1_opens,wk4_opens,mo3_opens,mo6_opens,yr1_opens,wk1_clicks,wk4_clicks,mo3_clicks,mo6_clicks,rev_dec,rev_raw,snds_dec,snds_opens_dec,wk1_vis,wk4_vis,mo3_vis,mo6_vis,wk1_pv,wk4_pv,mo3_pv,mo6_pv,wk1_secs,wk4_secs,mo3_secs,mo6_secs,prev_optouts,last_order,last_click,last_cart,last_visit,last_ded_open,sent_time,offer_signature_id,dynamic_content_signature_id,message_size,open_cnt,first_open_time,last_open_time,hrs_to_first_open,hrs_to_last_open,click_cnt,first_click_time,last_click_time,offer_category,hrs_to_first_click,hrs_to_last_click,opt_out_cnt,first_opt_out_time,last_opt_out_time,opt_out_source,opt_out_reason,hrs_to_first_opt_out,hrs_to_last_opt_out,campaign_id,subject,marketing_strategy,campaign_type
0,251392782,2019-08-11,175407402,Checkout_Online,1,0,48.98000,379,7,6,Non-Holiday,16,0,0,1,4,9,0,0,0,1,0.19021,128.97000,6.49108,0.22946,0,0,0,0,0,0,0,0,0,0,0,0,0,378.00000,175.00000,nan,nan,35.00000,2019-08-11 17:11:15,581429320,nan,149211,1.00000,2019-08-11 19:34:58,2019-08-11 19:34:58,2.00000,2.00000,1.00000,2019-08-11 19:35:17,2019-08-11 19:35:17,Mens_Pants_Straight,2.00000,2.00000,nan,NaN,NaN,NaN,NaN,nan,nan,59123842,It’s almost the end,Dockers_Adhoc_Promotional,20190811_US_Dockers_Tops_GraphicTees_30off_FS_...
1,273890682,2019-12-26,176031042,Footer_Desktop,0,0,27.74000,-72,3,6,Non-Holiday,0,0,0,0,0,0,0,0,0,0,nan,nan,nan,nan,0,0,0,0,0,0,0,0,0,0,0,0,0,nan,nan,nan,nan,nan,2020-03-14 19:43:46,-1610606741,224451348.00000,111097,nan,NaN,NaN,nan,nan,nan,NaN,NaN,NaN,nan,nan,nan,NaN,NaN,NaN,NaN,nan,nan,59253142,"<#include ""cms://messagelibrary/email/59253142...",Dockers_Program_WelcomeGeneral,DOC_WelNoPurchT4_Trigger_20191226
2,273890682,2019-12-26,176031082,Footer_Desktop,0,0,27.74000,-72,3,6,Non-Holiday,0,0,0,0,0,0,0,0,0,0,nan,nan,nan,nan,0,0,0,0,0,0,0,0,0,0,0,0,0,nan,nan,nan,nan,nan,2020-03-12 18:43:46,1329572970,746908335.00000,175946,nan,NaN,NaN,nan,nan,nan,NaN,NaN,NaN,nan,nan,nan,NaN,NaN,NaN,NaN,nan,nan,59253042,"<#include ""cms://messagelibrary/email/59253042...",Dockers_Program_WelcomeGeneral,DOC_WelNoPurchT3_Trigger_20191226
3,273890682,2019-12-26,176031062,Footer_Desktop,1,0,27.74000,-72,3,6,Non-Holiday,0,0,0,0,0,0,0,0,0,0,nan,nan,nan,nan,0,0,0,0,0,0,0,0,0,0,0,0,0,nan,nan,nan,nan,nan,2020-03-07 18:20:54,-2128439724,746908335.00000,163449,2.00000,2020-03-07 19:54:09,2020-03-07 20:47:41,1.00000,2.00000,nan,NaN,NaN,NaN,nan,nan,nan,NaN,NaN,NaN,NaN,nan,nan,59251982,"<#include ""cms://messagelibrary/email/59251982...",Dockers_Program_WelcomeGeneral,DOC_WelNoPurchT1_Trigger_20191226
4,273890682,2019-12-26,176031022,Footer_Desktop,0,0,27.74000,-72,3,6,Non-Holiday,0,0,0,0,0,0,0,0,0,0,nan,nan,nan,nan,0,0,0,0,0,0,0,0,0,0,0,0,0,nan,nan,nan,nan,nan,2020-03-10 17:43:46,2066955832,746908335.00000,228854,nan,NaN,NaN,nan,nan,nan,NaN,NaN,NaN,nan,nan,nan,NaN,NaN,NaN,NaN,nan,nan,59252822,"<#include ""cms://messagelibrary/email/59252822...",Dockers_Program_WelcomeGeneral,DOC_WelNoPurchT2_Trigger_20191226
5,260343962,2019-11-29,175893902,Lightbox_Mobile,1,0,59.98000,355,12,0,Non-Holiday,53,0,0,0,0,1,0,0,0,0,nan,nan,1.46535,0.01291,0,0,0,0,0,0,0,0,0,0,0,0,0,nan,nan,nan,nan,nan,2019-11-29 13:07:15,230762889,nan,163400,2.00000,2019-11-29 16:30:49,2019-11-29 17:01:18,3.00000,4.00000,2.00000,2019-11-29 16:31:06,2019-11-29 17:01:35,"Mens, Sale",3.00000,4.00000,nan,NaN,NaN,NaN,NaN,nan,nan,59216502,50% off absolutely everything + free shipping,Dockers_Adhoc_Promotional,20191129_US_Dockers_Dedicated_BF_50Off_FSFR_AM...
6,206326602,2019-11-29,175893902,My Account_Desktop,1,0,39.00000,1253,6,5,Non-Holiday,0,1,0,0,3,3,0,0,0,1,nan,nan,7.06333,1.05217,0,0,0,0,0,0,0,0,0,0,0,0,0,nan,160.00000,nan,nan,194.00000,2019-11-29 13:08:27,230762889,nan,164971,1.00000,2019-11-29 16:42:59,2019-11-29 16:42:59,3.00000,3.00000,3.00000,2019-11-29 16:43:02,2019-11-29 18:13:32,"Promotion, Mens_ShirtTops, Brand",3.00000,5.00000,nan,NaN,NaN,NaN,NaN,nan,nan,59216502

# Finalizing the data preprocessing

First, we work on this small data snippet and finalise the steps required for cleaning/preprocessing. 

Then, we iterate over each chunk, chain the preprocessing steps to each chunk and keep appending them to an accumulating dataframe.

**QUESTION:** How do we split the train-valid-test set?

1.   For each of the provided files, we employ a 80-20 split.
2.   For the full set of files, we employ a 80-20 split. i.e., the full first 2 files and a portion of the first half of the last file is used for training and remaining is used for validation.



In [ ]:
#Too many questions around what to keep, what to drop...
#Hence, currently default behaviour of drop
#Using a control variable dictionary to modify behaviour for relevant column groups
control_variable = {'aq': True,'site': True,'last':True, 'promo_ids': True,
                    'email_cnts':True, 'email_times':True, 'opt_out':True}

In [21]:
"""
Drop less-frequent AND/OR seemingly redundant columns
all team members voted to eliminate
"""

#Columns that are finalised to be dropped
df_sample = df.copy()
col_drop_names = ["send_dt", "sub_sourcedev", "rev_dec", "rev_raw", "snds_dec", "snds_opens_dec", 
            "dynamic_content_signature_id"]

#Memory footprint of the original sample is 53MB
df_sample.drop(columns=col_drop_names, inplace=True) #40MB
del col_drop_names

"""
Compute the composite recency, frequency & retention scores
Dropping the constituent base columns

NOTE 
----
1) Modified recency score to incorporate (1) click data and (2) yr_1 opens
2) Both recency and frequency are strongly correlated
3) sends_since_last_open used for retention score AS WELL AS reward
"""

df_sample['retention_score'] = df_sample.sends_since_last_open.apply(lambda x: 28/max(1,x)) 

#Modified recency_score a bit to incorporate clicks cos might as well...
df_sample["recency_score"] = 16/7*(df_sample['wk1_opens'] + df_sample['wk1_clicks']) 
+ 8/28*(df_sample['wk4_opens'] + df_sample['wk4_clicks']) 
+ 4/64*(df_sample['mo3_opens'] + df_sample['mo3_clicks']) 
+ 2/92*(df_sample['mo6_opens'] + df_sample['mo6_clicks']) 
+ 1/184*df_sample['yr1_opens']

#Will frequency score not be quite correlated with recency? That's what pandas profiling also was showing
df_sample["frequency_score"] = df_sample['wk1_opens'] + df_sample['wk4_opens'] + \
          df_sample['mo3_opens'] + df_sample['mo6_opens']

#Drop the base columns
col_drop_scores = ["wk1_opens", "wk4_opens", "mo3_opens","mo6_opens","yr1_opens",
                   "wk1_clicks", "wk4_clicks", "mo3_clicks", "mo6_clicks"]

#Memory footprint was 40MB
df_sample.drop(columns=col_drop_scores, inplace=True) #36MB
del col_drop_scores

[INFO] Dropped 16 from the snippet: 53 columns left


In [ ]:
"""
Acquisiton Columns - "aq_dt", "aq_mo", "aq_dow", "aq_period"

NOTE
----
1) Older customers who have not subscribed — exploit strategy should work better
2) aq_dt has negative numbers - Why?
"""
if control_variable["aq"]:
  col_drop_aq = ["aq_dt", "aq_mo", "aq_dow", "aq_period"]
  df_sample.drop(columns=col_drop_aq, inplace=True)

In [ ]:
"""
Website Data - "wk1_vis", "wk4_vis", "mo3_vis", "mo6_vis", "wk1_pv",
"wk4_pv", "mo3_pv", "mo6_pv", "wk1_secs", "wk4_secs", "mo3_secs", "mo6_secs"

NOTE
----
1) Extremely sparse - mostly filled with 0s
2) Marko had said website and email integration has not taken place
3) Naturally, this integration will happen in the future. Let the future team take care of it then.
"""
if control_variable["site"]:
  col_drop_site = ["wk1_vis", "wk4_vis", "mo3_vis", "mo6_vis", "wk1_pv",
"wk4_pv", "mo3_pv", "mo6_pv", "wk1_secs", "wk4_secs", "mo3_secs", "mo6_secs"]
  df_sample.drop(columns=col_drop_site, inplace=True)

In [ ]:
"""
The Last Time - "last_order", "last_click", "last_cart","last_visit","last_ded_open"

NOTE
----
1) Moooostly NaN values currently
2) However, when it is not NaN - especially for last_order - it could be a really useful signal
"""
if control_variable["last"]:
  col_drop_last = ["last_order", "last_click", "last_cart","last_visit","last_ded_open"]
  df_sample.drop(columns=col_drop_last, inplace=True)

In [ ]:
"""
Promotion IDs - "offer_signature_id", "launch_id", "campaign_id"

NOTE
----
1) A bunch of strings - might be useful for either cross-feature interactions or 
campaign/promotion embeddings
2) I feel it is a nice-to-have thing, as opposed to a need-to-have-thing
"""
if control_variable["promo_ids"]:
  col_drop_promo_ids = ["offer_signature_id", "launch_id", "campaign_id"]
  df_sample.drop(columns=col_drop_promo_ids, inplace=True)

In [24]:
"""
Detailed user-email interaction COUNTS - "open_cnt", "click_cnt" 
""
NOTE
----
1) We already have the "opened" signal - can it be boosted further with open_cnt and click_cnt?
2) But only 28.2% and 8.3% of these counts are available
"""
if control_variable["email_cnts"]:
  col_drop_email_cnts = ["open_cnt", "click_cnt"]
  df_sample.drop(columns=col_drop_email_cnts, inplace=True)

'\nDetailed user-email interaction COUNTS - "open_cnt", "click_cnt", "opt_out_cnt\n\nNOTE\n----\n\n'

In [ ]:
"""
Detailed user-email interaction TIMES - "first_open_time", "last_open_time", "hrs_to_first_open", 
"hrs_to_last_open", "first_click_time", "last_click_time", "hrs_to_first_click", "hrs_to_last_click"

NOTE
----
1) Hmmmm...
"""
if control_variable["email_times"]:
  col_drop_email_times = ["first_open_time", "last_open_time", "hrs_to_first_open", 
"hrs_to_last_open", "first_click_time", "last_click_time", "hrs_to_first_click", "hrs_to_last_click"]
  df_sample.drop(columns=col_drop_email_times, inplace=True)

In [ ]:
"""
Opt Out - "first_opt_out_time", "opt_out_cnt", "last_opt_out_time", "opt_out_source", 
"opt_out_reason", "hrs_to_first_opt_out", "hrs_to_last_opt_out"

NOTE
-----
1) Marko's response: so I think there's a couple of different levels you can opt out on, just promotional emails, all emails etc.
so they probably resubscribed if they got added in later
2) I think the emailer system has checks and balances to see whether people who opt out,
should be sent mails to. But they don't keep track of the past - i.e., when someone opts out,
then resubscribes - he/she does not get counted as an opt out
3) But only, 0.3% of entries have opt out filled
"""
if control_variable["opt_out"]:
  col_drop_opt_out = ["first_opt_out_time", "opt_out_cnt", "last_opt_out_time", "opt_out_source", 
"opt_out_reason", "hrs_to_first_opt_out", "hrs_to_last_opt_out"]
  df_sample.drop(columns=col_drop_opt_out, inplace=True)


In [10]:
cnt = 0
reader = pd.read_csv(file_name_l[0], sep='\t', header=None, names=col_names, chunksize=1e5)